# Предсказание бойца - победителя UFC
## Моделирование
### Межгалактический хакатон 2022
Команда "Meldonium" <br/>
Состав: Вячеслав Барков, Павел Мамаев, Сергей Глуховский, Алексей Недоливко, Андрей Рем, Иван Ершов


In [1]:
!pip install autogluon==0.4.1b20220516

In [2]:
import pandas as pd
import numpy as np
import pickle
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
path = 'events_df.bin'
with open(path, 'rb') as f:
    data  = pickle.load(f)

Загрузим наши данные, отсортируем и исключим лишние данные

In [4]:
data = data.sort_values(by=['eventDate.date'])
test_with_odds = data.iloc[-900:].reset_index(drop=True)
data = data.drop(columns=['eventDate.date', 'f1_odds', 'f2_odds'])
mask = (data['f1_n_fights'] != 1) & (data['f2_n_fights'] != 1)
data = data[mask]

Выделим валидационную и тестовую выборки

In [5]:
train = data.iloc[:-1800].reset_index(drop=True)
val = data.iloc[-1800:-900].reset_index(drop=True)
test = data.iloc[-900:].reset_index(drop=True)

Для создания модели будем использовать фреймворк AutoGluon. Это поможет нам  создать модель с высокой точностью и оптимизированными гиперпараметрами


In [6]:
predictor = TabularPredictor(label='winner', path='models', eval_metric='f1')
presets = ['best_quality']
predictor.fit(train, tuning_data=val, presets=presets, use_bag_holdout=True)

Presets specified: ['best_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "models/"
AutoGluon Version:  0.4.1b20220516
Python Version:     3.8.6
Operating System:   Darwin
Train Data Rows:    4372
Train Data Columns: 42
Tuning Data Rows:    900
Tuning Data Columns: 42
Label Column: winner
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [False, True]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = True, class 0 = False
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5747.6 MB
	Train Data (Original)  Memory Usage: 1.77 MB (0.0% of available memory)
	Inferring data type of each feature 

Fitting model: NeuralNetTorch_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.7209	 = Validation score   (f1)
	35.54s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Fitting model: WeightedEnsemble_L2 ...
	0.749	 = Validation score   (f1)
	1.17s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 11 L2 models ...
Fitting model: LightG

Fitting model: NeuralNetTorch_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.754	 = Validation score   (f1)
	29.78s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Fitting model: WeightedEnsemble_L3 ...
	0.7606	 = Validation score   (f1)
	0.86s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1

Посмотрим результаты нащих моделей проверив их на отложенной тестовой выборке

In [7]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,0.680031,0.738654,0.106249,0.204554,0.916495,0.106249,0.204554,0.916495,1,True,4
1,RandomForestGini_BAG_L1,0.675969,0.732812,0.100218,0.207016,1.899971,0.100218,0.207016,1.899971,1,True,3
2,ExtraTreesGini_BAG_L1,0.675112,0.720827,0.135555,0.231619,0.642405,0.135555,0.231619,0.642405,1,True,6
3,ExtraTreesEntr_BAG_L1,0.673637,0.716925,0.152326,0.216197,0.614457,0.152326,0.216197,0.614457,1,True,7
4,CatBoost_BAG_L1,0.673108,0.744072,0.021654,0.027891,11.751707,0.021654,0.027891,11.751707,1,True,5
5,NeuralNetTorch_BAG_L2,0.672609,0.753988,1.378220,2.423287,112.999387,0.197301,0.152415,29.779418,2,True,17
6,WeightedEnsemble_L2,0.671937,0.748991,0.231482,0.442275,15.741631,0.003361,0.002814,1.173458,2,True,10
7,WeightedEnsemble_L3,0.670000,0.760588,2.019247,3.447344,157.831899,0.003448,0.002471,0.856376,3,True,18
8,CatBoost_BAG_L2,0.668333,0.749568,1.200085,2.306385,94.974629,0.019166,0.035513,11.754660,2,True,13
9,ExtraTreesEntr_BAG_L2,0.665012,0.740051,1.302809,2.548088,83.954711,0.121890,0.277216,0.734742,2,True,15


Проверим, какие факторы больше всего влияют на точность нашей модели

In [8]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 42 features using 900 rows with 5 shuffle sets...
	435.15s	= Expected runtime (87.03s per shuffle set)
	148.93s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
f2_n_fights,0.034135,0.010849,1.075379e-03,5,0.056473,0.011797
f2_avg_fight_time,0.031882,0.005941,1.382197e-04,5,0.044114,0.019649
f1_n_fights,0.028275,0.001141,3.175081e-07,5,0.030624,0.025926
f2_age,0.012360,0.001963,7.383414e-05,5,0.016402,0.008319
f1_avg_fight_time,0.010725,0.004016,1.974725e-03,5,0.018994,0.002457
f2_win_ko_per_fight,0.010559,0.004446,3.022015e-03,5,0.019713,0.001405
f1_height,0.008806,0.003096,1.566554e-03,5,0.015181,0.002431
f2_win_sub_per_fight,0.005776,0.003388,9.447166e-03,5,0.012752,-0.001199
f1_win_ko_per_fight,0.005336,0.004738,3.274681e-02,5,0.015092,-0.004421
f1_age,0.005320,0.001486,6.605657e-04,5,0.008380,0.002260


Сохраним предсказания лучшей модели для дальнейшей работы с ними

In [14]:
test_with_odds['prediction'] = predictor.predict(test, model='LightGBMLarge_BAG_L1')

In [15]:
path = 'predictions.bin'
with open(path, 'wb') as f:
    pickle.dump(test_with_odds, f)